# **Lab 1: Matrix factorization**
**Matteus Berg**

# **Abstract**


The aim of this lab report was to implement algorithms for
* sparse matrix multiplication
* gram schmidt $QR$ factorization
* solving the matrix equation $Ax = b$

To do this, four methods in python were written. Correctness was ensured by testing the implemented methods.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [ ]:
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.



'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell.

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np


from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import csr_array

# **Introduction**

Today, matricies and vectors lie at the core of describing data for problems in computational science. Many real world problems consists of solving the equation

$Ax = b$

Where A is a nonsingular $nxn$ matrix, b and x are column vectors of size $n$. $b$ and $A$ are known, $x$ is unknown. To solve this equation, we have to invert $A$ so we get the following expression

$x = A^{-1}b$

However, for most matricies, inverting them is not a simple task. To make inversion easier, we first have to factor $A$ into matricies that are easy to invert. For this lab, we will use [modified gram-schmidt](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) $QR$ factorization.

$A = QR$

We will use the modified gram-schmidt iteration mentioned on page 89 of the book. This is instead of the regular gram-schmidt, since the regular one is prone to rounding errors when implemented on a computer.

Additionally, this lab will implement sparse matrix-vector multiplication. The matrix will be on Compressed Row Format (CRS). Compressing sparse matricies is benefitial for the computational cost. Regular matrix-vector multiplication results in $O(n^2)$ complexity, whereas with sparse methods you get $O(n)$.


# **Method**

Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.  



In [ ]:
# sparse matrix-vector product
def sparse_matrix_vector_product(A: csr_array, x: np.ndarray):
  n = A.shape[1]
  b = np.zeros(n)
  for i in range(n):
    for j in range(A.indptr[i], A.indptr[i+1]):
      b[i] = b[i] + A.data[j]*x[A.indices[j]]

  return b

# modified gram schmidt iteration
def modified_gram_schmidt_iteration(A):
  n = A.shape[0]
  R = np.zeros(A.shape);
  Q = np.zeros(A.shape);
  for j in range(0,n):
    v = A[:,j]
    for i in range(0, j):
      R[i,j] = np.dot(Q[:,i], v)
      v = v - R[i,j]*Q[:,i]
    R[j,j] = np.linalg.norm(v)
    Q[:,j] = v/R[j,j]

  return Q, R


# Matrix equation solve
def matrix_equation_solve(A: np.ndarray, b: np.ndarray):
  n = A.shape[0]
  x = np.copy(b)
  Q, R = modified_gram_schmidt_iteration(A)
  Qinv = np.transpose(Q)
  x = np.matmul(Qinv, x)
  #print(x)
  x = backward_substitution(R, x)
  #print(x)

  return x

def backward_substitution(U: np.ndarray, b: np.ndarray):
  n = U.shape[0]
  x = np.zeros_like(b)
  x[n-1] = b[n-1]/U[n-1,n-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i+1, n, 1):
      sum = sum + U[i,j]*x[j]
    x[i] = (b[i] - sum)/U[i,i]

  return x


# **Results**

Running of the three algorithms is done below. Output is the output of the tests.

In [ ]:
val = np.array([3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3]) # data
col_idx = np.array([0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5]) # indices
row_ptr = np.array([0, 3, 5, 6, 8, 9, 11]) # indptr
A = csr_array((val, col_idx, row_ptr), shape=(6, 6))
x = np.array([1, 1, 1, 1, 1, 1])
b = sparse_matrix_vector_product(A, x)

A2 = [[3,2,0,2,0,0], [0,2,1,0,0,0], [0,0,1,0,0,0], [0,0,3,2,0,0], [0,0,0,0,1,0], [0,0,0,0,2,3]]
b2 = np.matmul(A2, x)

print("Sparse matrix-vector multiplication")
print("Sparse matrix multiplication result: ", b)
print("Dense matrix multiplication result: ", b2)

A = np.array([[2,-1], [-1,2]]);
Q, R = modified_gram_schmidt_iteration(A)

print("\nModified Gram Schmidt iteration")
print("Q:\n", Q, "\nR:\n", R)
print("Frobenius norm Q^TQ-I: ", (np.linalg.norm(np.matmul(np.transpose(Q), Q) - np.identity(2))))
print("Frobenius norm QR-A ", (np.linalg.norm(np.matmul(Q, R) - A)))

A = np.array([[2,-1], [-1,2]]);
b = np.array([5,-4])
x = matrix_equation_solve(A, b)
y = np.array([2, -1])
print("\nMatrix equation tests.")
print("value of x: ", x)
print("Matrix equation residual ||Ax-b||: ", np.matmul(A, x) - b)
print("Matrix equation residual ||x-y|| (b=Ay):", x - y)

Sparse matrix-vector multiplication
Sparse matrix multiplication result:  [7. 3. 1. 5. 1. 5.]
Dense matrix multiplication result:  [7 3 1 5 1 5]

Modified Gram Schmidt iteration
Q:
 [[ 0.89442719  0.4472136 ]
 [-0.4472136   0.89442719]] 
R:
 [[ 2.23606798 -1.78885438]
 [ 0.          1.34164079]]
Frobenius norm Q^TQ-I:  2.6901577681355055e-16
Frobenius norm QR-A  0.0

Matrix equation tests.
value of x:  [ 2. -1.]
Matrix equation residual ||Ax-b||:  [-8.8817842e-16 -8.8817842e-16]
Matrix equation residual ||x-y|| (b=Ay): [-8.8817842e-16 -8.8817842e-16]


# **Discussion**

The results give what was expected. The sparse matrix and dense matrix multiplications are identical. The first frobenious norm is almost zero. The reason for it not being zero is probably due to rounding errors. For the matrix equation solver, we get the correct solution for x. The residuals aren't zero, but that is once again due to rounding errors.